In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030_time_cross_80.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 108 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(108): SRR7817623 SRR7817626 ... SRR3593573 SRR7817701
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [6]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 108
                   SRR7817623 SRR7817626 SRR7817614 SRR7817675 SRR1636593
ENSMUSG00000000001  15.506769  15.687526  15.624661  15.709312  16.323003
ENSMUSG00000000028   7.827669   7.638854   7.756799   7.916277   8.279175
ENSMUSG00000000049  20.408033  20.329207  20.475516  20.044106  20.510324
ENSMUSG00000000058   8.642803   8.553315   8.715385   8.982900   8.807459
ENSMUSG00000000085  10.425142  10.697880  10.766237  10.004847  10.316191
ENSMUSG00000000088  16.324784  16.551933  16.864250  16.657197  15.996544
                   SRR7817627 SRR3593526 SRR7817639 SRR7817615 SRR7817684
ENSMUSG00000000001  15.483115  14.883153  15.520394  15.627692  15.707271
ENSMUSG00000000028   8.392259   7.981391   7.614392   7.495814   7.606281
ENSMUSG00000000049  20.354301  20.947744  20.262921  20.388134  20.484448
ENSMUSG00000000058   8.500589   8.701991   8.631645   8.260848   8.303304
ENSMUSG00000000085  10.285214  10.891720  10.901856  10.679014  10.009943
ENSMUSG00000000088  16.735641 

ENSMUSG00000000085  11.192994  10.927759  10.737232  11.168156   11.04483
ENSMUSG00000000088  16.543799  16.282824  16.579715  16.468166   16.36482
                   SRR7817621 SRR7817653 SRR7817691 SRR3593574 SRR1636672
ENSMUSG00000000001  15.953973  16.165455  16.676660  15.212080  16.504336
ENSMUSG00000000028   8.522215   8.559549   8.394826   8.594033   7.777827
ENSMUSG00000000049  17.308901  17.778489  17.732787  18.832244  19.860770
ENSMUSG00000000058  10.869497   9.473571   9.703098   9.516907  10.524088
ENSMUSG00000000085  10.924393  10.660352  11.098436  10.387353  10.235136
ENSMUSG00000000088  16.483702  16.350012  16.432346  15.718890  15.898556
                   SRR4317655 SRR1636675 SRR3593577 SRR7817705 SRR4317657
ENSMUSG00000000001  15.946659  16.523622  15.136968  16.538219  16.102410
ENSMUSG00000000028   8.629218   8.120658   8.324397   8.949931   9.029317
ENSMUSG00000000049  17.455677  20.358074  19.579259  17.848309  17.893036
ENSMUSG00000000058  10.886394  10.0067

In [7]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030_time_cross_80.Rds", sep = ""))
print(tcdd_dose_detail.vec)

  [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [26]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [51]  0  0  0  0 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 [76] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[101] 30 30 30 30 30 30 30 30


In [8]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030_time_cross_80.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000039740"

$`R-MMU-448741`
[1] "ENSMUSG00000039760" "ENSMUSG00000074695"

$`R-MMU-71200`
[1] "ENSMUSG00000039783"

$`R-MMU-8854255`
[1] "ENSMUSG00000039813" "ENSMUSG00000079477"

$`R-MMU-442319`
[1] "ENSMUSG00000039878"

$`R-MMU-8875871`
[1] "ENSMUSG00000039908"

$`R-MMU-8879117`
[1] "ENSMUSG00000039952" "ENSMUSG00000066235"

$`R-MMU-9018768`
[1] "ENSMUSG00000039960"

$`R-MMU-9726862`
[1] "ENSMUSG00000039960"

$`R-MMU-391933`
[1] "ENSMUSG00000040016"

$`R-MMU-2995334`
[1] "ENSMUSG00000040018"

$`R-MMU-450494`
[1] "ENSMUSG00000040028"

$`R-MMU-450550`
[1] "ENSMUSG00000040028"

$`R-MMU-517674`
[1] "ENSMUSG00000040028"

$`R-MMU-517705`
[1] "ENSMUSG00000040028"

$`R-NUL-5362406`
[1] "ENSMUSG00000040035"

$`R-MMU-139909`
[1] "ENSMUSG00000040093"

$`R-MMU-8952137`
[1] "ENSMUSG00000040105"

$`R-MMU-9650858`
[1] "ENSMUSG00000040136" "ENSMUSG00000096146"

$`R-MMU-9652580`
[1] "ENSMUSG00000040136" "ENSMUSG00000096146"

$`R-MMU-141200`
[1] "ENSMUSG00000040147"

$`R-MMU-141202`
[1] "EN

[1] "ENSMUSG00000042854" "ENSMUSG00000059552"

$`R-MMU-9614414`
[1] "ENSMUSG00000042903" "ENSMUSG00000044167" "ENSMUSG00000048756"

$`R-MMU-9617657`
[1] "ENSMUSG00000042903"

$`R-MMU-9617711`
[1] "ENSMUSG00000042903"

$`R-MMU-9617755`
[1] "ENSMUSG00000042903"

$`R-NUL-9617710`
[1] "ENSMUSG00000042903"

$`R-NUL-9617742`
[1] "ENSMUSG00000042903"

$`R-NUL-9617757`
[1] "ENSMUSG00000042903"

$`R-NUL-9617903`
[1] "ENSMUSG00000042903"

$`R-NUL-9617912`
[1] "ENSMUSG00000042903"

$`R-MMU-187246`
[1] "ENSMUSG00000043013" "ENSMUSG00000045518"

$`R-MMU-189119`
[1] "ENSMUSG00000043013" "ENSMUSG00000045518"

$`R-MMU-391942`
[1] "ENSMUSG00000043017"

$`R-MMU-9726949`
[1] "ENSMUSG00000043017"

$`R-MMU-1564169`
[1] "ENSMUSG00000043089" "ENSMUSG00000056174" "ENSMUSG00000068196"

$`R-MMU-1602473`
[1] "ENSMUSG00000043089"

$`R-MMU-389424`
[1] "ENSMUSG00000043102"

$`R-MMU-9014672`
[1] "ENSMUSG00000043241" "ENSMUSG00000058301"

$`R-MMU-9014678`
[1] "ENSMUSG00000043241" "ENSMUSG00000058301"

$`R-MMU-162873`

[1] "ENSMUSG00000046971" "ENSMUSG00000050211" "ENSMUSG00000056220"
[4] "ENSMUSG00000070719" "ENSMUSG00000098488"

$`R-MMU-1482897`
[1] "ENSMUSG00000046971" "ENSMUSG00000056220" "ENSMUSG00000070719"

$`R-MMU-1482920`
[1] "ENSMUSG00000046971" "ENSMUSG00000070719"

$`R-MMU-9794031`
[1] "ENSMUSG00000047002"

$`R-MMU-437240`
[1] "ENSMUSG00000047139"

$`R-MMU-140355`
[1] "ENSMUSG00000047250"

$`R-MMU-140359`
[1] "ENSMUSG00000047250"

$`R-MMU-2314678`
[1] "ENSMUSG00000047250"

$`R-MMU-9677320`
[1] "ENSMUSG00000047250"

$`R-MMU-6803890`
[1] "ENSMUSG00000047281"

$`R-MMU-947531`
[1] "ENSMUSG00000047454"

$`R-MMU-5340195`
[1] "ENSMUSG00000047638"

$`R-MMU-6806674`
[1] "ENSMUSG00000047719"

$`R-MMU-190681`
[1] "ENSMUSG00000047797"

$`R-MMU-9846477`
[1] "ENSMUSG00000047878"

$`R-MMU-8850846`
[1] "ENSMUSG00000048120"

$`R-MMU-8850854`
[1] "ENSMUSG00000048120"

$`R-MMU-8954468`
[1] "ENSMUSG00000048142"

$`R-MMU-70542`
[1] "ENSMUSG00000048217"

$`R-MMU-6804402`
[1] "ENSMUSG00000048349" "ENSMUSG000000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

In [10]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [11]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108


In [12]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

  [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [26]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [51]  0  0  0  0 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
 [76] 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
[101] 30 30 30 30 30 30 30 30


In [13]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                   SRR7817623 SRR7817626 SRR7817614 SRR7817675 SRR1636593
ENSMUSG00000000001  15.506769  15.687526  15.624661  15.709312  16.323003
ENSMUSG00000000028   7.827669   7.638854   7.756799   7.916277   8.279175
ENSMUSG00000000049  20.408033  20.329207  20.475516  20.044106  20.510324
ENSMUSG00000000058   8.642803   8.553315   8.715385   8.982900   8.807459
ENSMUSG00000000085  10.425142  10.697880  10.766237  10.004847  10.316191
ENSMUSG00000000088  16.324784  16.551933  16.864250  16.657197  15.996544
ENSMUSG00000000120   9.497220   9.379570  10.138771   8.577005   7.776667
ENSMUSG00000000126   7.404155   7.167072   7.073775   7.220467   6.755998
ENSMUSG00000000127  10.193392   9.871816  10.041260   9.395139  10.569879
ENSMUSG00000000142  10.983382   9.976326  10.101271   9.017756   9.951001
ENSMUSG00000000149  15.919898  15.704915  16.032148  16.168641  14.908434
ENSMUSG00000000154  15.752836  15.475192  15.566108  15.648031  15.268476
ENSMUSG00000000168  15.201222  14.4472

ENSMUSG00000001105  13.365610  13.734487  13.342058  13.492410  12.950173
ENSMUSG00000001119   9.465417   9.522390   9.621606   9.221006   9.705917
ENSMUSG00000001123  18.690119  18.616438  18.677915  17.503848  18.546674
ENSMUSG00000001127  14.989465  15.150382  15.266274  15.143065  14.721707
ENSMUSG00000001128  12.852529  10.648657  11.606457  11.794933  10.785509
ENSMUSG00000001131   5.694439   5.703665   5.676790   6.654057   6.479502
ENSMUSG00000001143  11.028067  11.606838  11.889910  11.853275  11.006130
ENSMUSG00000001155  19.121244  18.968585  19.066981  18.767010  18.234305
ENSMUSG00000001158  11.424330  10.953636  11.189541  10.517199  11.441367
ENSMUSG00000001173  10.388993  10.599248  10.370214   9.636472  11.000219
ENSMUSG00000001175  17.086488  17.139925  17.496640  17.366475  16.765832
ENSMUSG00000001211  16.946310  17.079518  17.067544  16.917500  16.259167
ENSMUSG00000001225   6.828655   5.703665   5.676790   5.695567   5.715567
ENSMUSG00000001228   6.963683   5.7036

ENSMUSG00000002249  10.650196  10.924917  11.246101  10.547228  10.117065
ENSMUSG00000002250  10.694920  11.778841  13.767489  10.654031  10.046964
ENSMUSG00000002257   7.099950   7.738278   6.928275   7.096220   5.715567
ENSMUSG00000002279  13.083471  13.479562  13.531140  13.827829  12.808263
ENSMUSG00000002289  13.821926  14.286813  13.883617  14.407246  13.822839
ENSMUSG00000002297   8.411599   9.014357   8.296255   8.040064   8.141596
ENSMUSG00000002307  11.419877  11.284584  11.365500  11.169942  10.647388
ENSMUSG00000002308   9.607196   9.142413   9.078457   8.843715   9.301689
ENSMUSG00000002324   8.622877   8.046395   8.907031   8.271682   7.674708
ENSMUSG00000002325  13.734158  13.353478  13.550315  12.697735  12.880764
ENSMUSG00000002326  11.629203  11.501286  11.328390  11.062464  11.814708
ENSMUSG00000002346  15.745819  15.554190  15.899033  15.670445  15.091203
ENSMUSG00000002365  14.148156  13.783755  14.004376  14.047051  14.768032
ENSMUSG00000002379  13.314616  13.8189

ENSMUSG00000003379   7.382450   7.389977   6.557525   7.220467   7.114808
ENSMUSG00000003402  15.718404  15.800564  16.081745  15.899802  15.695284
ENSMUSG00000003411   7.076101   6.695418   6.857778   7.583947   6.755998
ENSMUSG00000003418   6.648284   6.431792   6.557525   6.649895   6.755998
ENSMUSG00000003421  12.305893  12.563194  12.747510  12.089285  11.449218
ENSMUSG00000003429  13.597926  14.251384  14.261297  14.088274  14.814701
ENSMUSG00000003435  14.515359  14.990876  14.669467  15.275400  14.104009
ENSMUSG00000003436   5.694439   5.703665   5.676790   5.695567   5.715567
ENSMUSG00000003437  13.100601  13.101107  13.308483  13.065037  13.163956
ENSMUSG00000003444   8.895720   9.443690   9.427949   9.696010   8.801862
ENSMUSG00000003452   6.648284   5.703665   5.676790   6.314795   6.755998
ENSMUSG00000003458  14.152933  13.996356  13.645630  13.895186  14.173740
ENSMUSG00000003464  15.833092  15.588766  15.861089  15.665873  16.066435
ENSMUSG00000003476   7.708407   5.7036

ENSMUSG00000004654   6.394652   5.703665   5.676790   5.695567   6.755998
ENSMUSG00000004655  14.711616  14.391887  14.789651  14.509835  14.734336
ENSMUSG00000004665   9.600662   9.332622   9.552983   9.382823   9.114489
ENSMUSG00000004667  12.297553  12.082624  12.905078  12.650635  11.792712
ENSMUSG00000004668   5.694439   5.703665   5.676790   5.695567   5.715567
ENSMUSG00000004677  13.216702  13.350456  13.114527  13.226123  12.088172
ENSMUSG00000004709   7.251198   5.703665   6.557525   6.981943   6.479502
ENSMUSG00000004730  11.745372  10.477188  10.091480  11.178184  10.640628
ENSMUSG00000004768  11.203012  11.017237  10.736416   9.895600  11.878829
ENSMUSG00000004771  14.430023  14.685772  14.580664  14.760218  14.382016
ENSMUSG00000004788  11.393008  12.243132  12.051970  12.014149  10.894620
ENSMUSG00000004789  16.780093  16.964617  17.163758  17.200675  17.280232
ENSMUSG00000004791   5.694439   5.703665   5.676790   6.399200   6.479502
ENSMUSG00000004815  13.949051  13.6857

...

ENSMUSG00000015355  10.636507  10.786721  10.196213  10.102460   9.820279
ENSMUSG00000015357  14.818457  14.113187  14.465955  14.995641  15.380468
ENSMUSG00000015377  10.784400  10.333200   9.737538  10.650166  10.013407
ENSMUSG00000015405   6.479890   7.017081   6.457338   6.579444   7.453018
ENSMUSG00000015437   7.484138   7.260382   6.956467   7.289696   6.798215
ENSMUSG00000015452   7.407097   7.476627   6.617078   8.014104   7.502279
ENSMUSG00000015461  13.277325  12.563640  13.092046  13.274216  12.975233
ENSMUSG00000015468  10.120456  10.352773   9.475726  11.074591   9.934074
ENSMUSG00000015474  13.234056  12.800699  13.110731  13.095474  13.182838
ENSMUSG00000015478  12.615415  12.238096  12.642731  12.289191  12.783437
ENSMUSG00000015522  12.947414  13.039665  13.595106  13.511518  13.449510
ENSMUSG00000015533   8.363439   9.837512   8.548246   9.699984   7.726771
ENSMUSG00000015568  16.746457  16.445656  17.391131  16.667147  16.423268
ENSMUSG00000015575  14.572938  14.6611

ENSMUSG00000001383  13.309372  14.683637  12.851453
ENSMUSG00000001403   8.013404   9.607646   7.671361
ENSMUSG00000001415  14.738817  13.627751  15.155351
ENSMUSG00000001416  15.188034  14.889254  15.102913
ENSMUSG00000001419  14.333505  13.079067  14.511521
ENSMUSG00000001435  16.665092  17.696780  16.745228
ENSMUSG00000001436  10.722987  10.304194  10.572807
ENSMUSG00000001440  16.447598  15.500954  16.452616
ENSMUSG00000001441  12.965258  12.147130  13.024104
ENSMUSG00000001445  14.113762  14.106296  14.185838
ENSMUSG00000001467  11.385094  15.241657  11.668973
ENSMUSG00000001473  12.001048  13.166856  12.333481
ENSMUSG00000001494   6.141048   5.664513   6.229905
ENSMUSG00000001504   5.632106   5.664513   5.653817
ENSMUSG00000001506  15.952887  16.320116  14.832410
ENSMUSG00000001507  12.586164  11.206338  12.633133
ENSMUSG00000001517   8.399712   9.315356   7.839382
ENSMUSG00000001518  10.946349  10.816509  11.402660
ENSMUSG00000001521   9.125241   8.526750   9.336052
ENSMUSG00000

ENSMUSG00000003037  13.612031  14.386372  13.810817
ENSMUSG00000003039  13.968649  14.160275  14.150532
ENSMUSG00000003051   8.228390   8.683970   8.610060
ENSMUSG00000003053  14.454343  18.920280  11.421904
ENSMUSG00000003062  10.480072  10.812970  10.722657
ENSMUSG00000003068  13.619323  15.891416  14.145926
ENSMUSG00000003070   6.136424   6.273401   6.700892
ENSMUSG00000003072  15.462589  17.140665  17.253474
ENSMUSG00000003099  12.644574  13.176408  13.488874
ENSMUSG00000003119  12.704063  11.756699  12.470976
ENSMUSG00000003131  14.101930  13.060618  14.218025
ENSMUSG00000003135  11.210881  11.918143  11.399511
ENSMUSG00000003153   7.664072   7.816336   8.285526
ENSMUSG00000003161  12.706179  13.052640  12.685324
ENSMUSG00000003184  12.875238  13.285868  13.444456
ENSMUSG00000003200  11.559244  12.116519  12.385558
ENSMUSG00000003206   7.652260   8.018548   8.060463
ENSMUSG00000003208   8.748595   9.327606   9.203245
ENSMUSG00000003226  16.769759  15.235112  16.120749
ENSMUSG00000

ENSMUSG00000004788  11.963004  12.046562  12.573027
ENSMUSG00000004789  17.503598  16.866362  17.693890
ENSMUSG00000004791   6.726397   7.397318   6.975942
ENSMUSG00000004815  13.170294  12.969181  13.151613
ENSMUSG00000004837  10.263212   9.764621  10.367255
ENSMUSG00000004843  14.120345  13.484295  14.299616
ENSMUSG00000004846  11.795853  12.249304  12.486347
ENSMUSG00000004849  13.798585  13.800284  13.890323
ENSMUSG00000004864   8.697545   9.784743   9.021178
ENSMUSG00000004865  14.429939  14.344960  13.578696
ENSMUSG00000004872   5.632106   5.664513   5.653817
ENSMUSG00000004880  11.251547  11.472113  10.715104
ENSMUSG00000004885   6.141048   6.273401   5.653817
ENSMUSG00000004892   6.141048   5.664513   5.925410
ENSMUSG00000004895  10.890412  11.161299  10.830092
ENSMUSG00000004902   7.305432   6.525297   7.103835
ENSMUSG00000004929  11.222042  10.532211  12.272478
ENSMUSG00000004933   8.556700   8.609470   9.037276
ENSMUSG00000004934  10.178673  11.162397  11.651933
ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

       Resample1
  [1,]         1
  [2,]         2
  [3,]         3
  [4,]         4
  [5,]         5
  [6,]         6
  [7,]         7
  [8,]         8
  [9,]         9
 [10,]        10
 [11,]        11
 [12,]        12
 [13,]        13
 [14,]        14
 [15,]        15
 [16,]        16
 [17,]        17
 [18,]        18
 [19,]        19
 [20,]        20
 [21,]        21
 [22,]        22
 [23,]        23
 [24,]        24
 [25,]        25
 [26,]        26
 [27,]        27
 [28,]        28
 [29,]        29
 [30,]        30
 [31,]        31
 [32,]        32
 [33,]        33
 [34,]        34
 [35,]        35
 [36,]        36
 [37,]        37
 [38,]        38
 [39,]        39
 [40,]        40
 [41,]        41
 [42,]        42
 [43,]        43
 [44,]        44
 [45,]        45
 [46,]        46
 [47,]        47
 [48,]        48
 [49,]        49
 [50,]        50
 [51,]        51
 [52,]        52
 [53,]        53
 [54,]        54
 [55,]        55
 [56,]        56
 [57,]        57
 [58,]        

In [15]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

                   SRR7817623 SRR7817626 SRR7817614 SRR7817675 SRR1636593
ENSMUSG00000000001  15.506769  15.687526  15.624661  15.709312  16.323003
ENSMUSG00000000028   7.827669   7.638854   7.756799   7.916277   8.279175
ENSMUSG00000000049  20.408033  20.329207  20.475516  20.044106  20.510324
ENSMUSG00000000058   8.642803   8.553315   8.715385   8.982900   8.807459
ENSMUSG00000000085  10.425142  10.697880  10.766237  10.004847  10.316191
ENSMUSG00000000088  16.324784  16.551933  16.864250  16.657197  15.996544
ENSMUSG00000000120   9.497220   9.379570  10.138771   8.577005   7.776667
ENSMUSG00000000126   7.404155   7.167072   7.073775   7.220467   6.755998
ENSMUSG00000000127  10.193392   9.871816  10.041260   9.395139  10.569879
ENSMUSG00000000142  10.983382   9.976326  10.101271   9.017756   9.951001
ENSMUSG00000000149  15.919898  15.704915  16.032148  16.168641  14.908434
ENSMUSG00000000154  15.752836  15.475192  15.566108  15.648031  15.268476
ENSMUSG00000000168  15.201222  14.4472

ENSMUSG00000001105  13.365610  13.734487  13.342058  13.492410  12.950173
ENSMUSG00000001119   9.465417   9.522390   9.621606   9.221006   9.705917
ENSMUSG00000001123  18.690119  18.616438  18.677915  17.503848  18.546674
ENSMUSG00000001127  14.989465  15.150382  15.266274  15.143065  14.721707
ENSMUSG00000001128  12.852529  10.648657  11.606457  11.794933  10.785509
ENSMUSG00000001131   5.694439   5.703665   5.676790   6.654057   6.479502
ENSMUSG00000001143  11.028067  11.606838  11.889910  11.853275  11.006130
ENSMUSG00000001155  19.121244  18.968585  19.066981  18.767010  18.234305
ENSMUSG00000001158  11.424330  10.953636  11.189541  10.517199  11.441367
ENSMUSG00000001173  10.388993  10.599248  10.370214   9.636472  11.000219
ENSMUSG00000001175  17.086488  17.139925  17.496640  17.366475  16.765832
ENSMUSG00000001211  16.946310  17.079518  17.067544  16.917500  16.259167
ENSMUSG00000001225   6.828655   5.703665   5.676790   5.695567   5.715567
ENSMUSG00000001228   6.963683   5.7036

ENSMUSG00000002249  10.650196  10.924917  11.246101  10.547228  10.117065
ENSMUSG00000002250  10.694920  11.778841  13.767489  10.654031  10.046964
ENSMUSG00000002257   7.099950   7.738278   6.928275   7.096220   5.715567
ENSMUSG00000002279  13.083471  13.479562  13.531140  13.827829  12.808263
ENSMUSG00000002289  13.821926  14.286813  13.883617  14.407246  13.822839
ENSMUSG00000002297   8.411599   9.014357   8.296255   8.040064   8.141596
ENSMUSG00000002307  11.419877  11.284584  11.365500  11.169942  10.647388
ENSMUSG00000002308   9.607196   9.142413   9.078457   8.843715   9.301689
ENSMUSG00000002324   8.622877   8.046395   8.907031   8.271682   7.674708
ENSMUSG00000002325  13.734158  13.353478  13.550315  12.697735  12.880764
ENSMUSG00000002326  11.629203  11.501286  11.328390  11.062464  11.814708
ENSMUSG00000002346  15.745819  15.554190  15.899033  15.670445  15.091203
ENSMUSG00000002365  14.148156  13.783755  14.004376  14.047051  14.768032
ENSMUSG00000002379  13.314616  13.8189

ENSMUSG00000003379   7.382450   7.389977   6.557525   7.220467   7.114808
ENSMUSG00000003402  15.718404  15.800564  16.081745  15.899802  15.695284
ENSMUSG00000003411   7.076101   6.695418   6.857778   7.583947   6.755998
ENSMUSG00000003418   6.648284   6.431792   6.557525   6.649895   6.755998
ENSMUSG00000003421  12.305893  12.563194  12.747510  12.089285  11.449218
ENSMUSG00000003429  13.597926  14.251384  14.261297  14.088274  14.814701
ENSMUSG00000003435  14.515359  14.990876  14.669467  15.275400  14.104009
ENSMUSG00000003436   5.694439   5.703665   5.676790   5.695567   5.715567
ENSMUSG00000003437  13.100601  13.101107  13.308483  13.065037  13.163956
ENSMUSG00000003444   8.895720   9.443690   9.427949   9.696010   8.801862
ENSMUSG00000003452   6.648284   5.703665   5.676790   6.314795   6.755998
ENSMUSG00000003458  14.152933  13.996356  13.645630  13.895186  14.173740
ENSMUSG00000003464  15.833092  15.588766  15.861089  15.665873  16.066435
ENSMUSG00000003476   7.708407   5.7036

ENSMUSG00000004654   6.394652   5.703665   5.676790   5.695567   6.755998
ENSMUSG00000004655  14.711616  14.391887  14.789651  14.509835  14.734336
ENSMUSG00000004665   9.600662   9.332622   9.552983   9.382823   9.114489
ENSMUSG00000004667  12.297553  12.082624  12.905078  12.650635  11.792712
ENSMUSG00000004668   5.694439   5.703665   5.676790   5.695567   5.715567
ENSMUSG00000004677  13.216702  13.350456  13.114527  13.226123  12.088172
ENSMUSG00000004709   7.251198   5.703665   6.557525   6.981943   6.479502
ENSMUSG00000004730  11.745372  10.477188  10.091480  11.178184  10.640628
ENSMUSG00000004768  11.203012  11.017237  10.736416   9.895600  11.878829
ENSMUSG00000004771  14.430023  14.685772  14.580664  14.760218  14.382016
ENSMUSG00000004788  11.393008  12.243132  12.051970  12.014149  10.894620
ENSMUSG00000004789  16.780093  16.964617  17.163758  17.200675  17.280232
ENSMUSG00000004791   5.694439   5.703665   5.676790   6.399200   6.479502
ENSMUSG00000004815  13.949051  13.6857

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  14.229963  14.123902  14.475059  14.329216  14.594526
ENSMUSG00000004070  13.723668  14.035891  13.543348  13.824059  13.651462
ENSMUSG00000004096  14.012318  13.284626  13.363123  13.377642  13.224069
ENSMUSG00000004098   9.709290   8.226702   7.958238   7.417018   7.799208
ENSMUSG00000004099  11.616015  12.404534  11.529055  11.433206  11.549511
ENSMUSG00000004110   7.219231   5.688822   6.260490   6.786627   7.074939
ENSMUSG00000004113   7.119730   7.477212   7.402291   6.646753   7.954686
ENSMUSG00000004187   8.153243   8.833598   9.233030   8.986911   9.681346
ENSMUSG00000004207  18.956343  20.133691  19.323469  19.391697  19.422215
ENSMUSG00000004221  16.794454  16.477290  16.622760  16.632443  16.610471
ENSMUSG00000004231   5.683199   5.688822   5.661338   5.664327   5.649903
ENSMUSG00000004263  11.520211  12.101858  12.328055  11.981273  12.623788
ENSMUSG00000004264  16.564477  16.359475  15.825403  16.056074  16.429317
ENSMUSG00000004266  13.103840  13.690230  12.855201  12.

ENSMUSG00000005299  14.664206  14.416129  14.750289  15.137322  15.141480
ENSMUSG00000005312  16.382424  16.114358  16.280279  16.456785  16.396249
ENSMUSG00000005320  11.524969  11.073471   9.740297   9.995189   9.687907
ENSMUSG00000005338   9.196697   9.009142   9.270766   8.691150   8.541876
ENSMUSG00000005339   5.683199   5.688822   5.661338   5.664327   5.649903
ENSMUSG00000005354  17.186701  17.303913  16.726970  16.990094  17.033320
ENSMUSG00000005355   5.683199   5.688822   5.661338   5.664327   5.649903
ENSMUSG00000005357   6.762041   6.365656   6.214537   6.225908   6.417273
ENSMUSG00000005360   6.349236   6.795388   6.741635   6.882316   6.679084
ENSMUSG00000005364   6.590589   6.753778   6.755355   6.492842   6.417273
ENSMUSG00000005370  10.960306  10.852861  10.895508  10.711876  10.630854
ENSMUSG00000005371  12.546108  12.085530  12.889347  12.595743  12.398194
ENSMUSG00000005373  16.417816  14.131239  13.876849  11.745234  13.449449
ENSMUSG00000005378  13.002326  12.7526

ENSMUSG00000006442  12.224269  12.624584  11.984984  12.679583  13.530109
ENSMUSG00000006445  12.216478  11.516937  11.163636  11.508026  12.169480
ENSMUSG00000006456  11.659234  11.694527  11.051253  11.300273  10.924170
ENSMUSG00000006457   6.349236   6.779664   6.477247   7.212090   5.649903
ENSMUSG00000006464   8.147766   7.968301   8.450895   8.585600   8.219542
ENSMUSG00000006469   5.683199   5.688822   5.661338   5.664327   5.649903
ENSMUSG00000006494  14.178441  13.261739  12.518966  12.227839  12.546620
ENSMUSG00000006498  15.143414  14.963164  15.281512  15.297477  15.323799
ENSMUSG00000006517  10.243063   8.902682   9.339278   9.352945   9.421630
ENSMUSG00000006519  11.174015  13.088799  11.971419  11.868966  11.889764
ENSMUSG00000006522  17.728694  17.544532  15.958373  15.619885  16.331436
ENSMUSG00000006538   7.127725   7.656162   7.486934   7.628816   7.632696
ENSMUSG00000006542   6.549943   6.573491   6.477247   6.225908   6.997550
ENSMUSG00000006567  12.442537  11.1834

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

In [17]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030_time_cross_80.Rds",sep=""))

In [18]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

In [19]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

In [20]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [21]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030_time_cross_80.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] "Processed 6200 of 9825 reactions (63.1%)..."
[1] "Processed 6300 of 9825 reactions (64.1%)..."
[1] "Processed 6400 of 9825 reactions (65.2%)..."
[1] "Processed 6500 of 9825 reactions (66.2%)..."
[1] "Processed 6600 of 9825 reactions (67.2%)..."
[1] "Processed 6700 of 9825 reactions (68.2%)..."
[1] "Processed 6800 of 9825 reactions (69.2%)..."
[1] "Processed 6900 of 9825 reactions (70.2%)..."
[1] "Processed 7000 of 9825 reactions (71.3%)..."
[1] "Storing PCA objects containing reactions 6000-7000 of 9825 reactions (71.3%)..."
[1] "Processed 7100 of 9825 reactions (72.3%)..."
[1] "Processed 7200 of 9825 reactions (73.3%)..."
[1] "Processed 7300 of 9825 reactions (74.3%)..."
[1] "Processed 7400 of 9825 reactions (75.3%)..."
[1] "Processed 7500 of 9825 reactions (76.3%)..."
[1] "Processed 7600 of 9825 reactions (77.4%)..."
[1] "Processed 7700 of 9825 reactions (78.4%)..."
[1] "Processed 7800 of 9825 reactions (79.4%)..."
[1] "Processed 7900 of 9825 reactions (80.4%)..."
[1] "Processed

In [22]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030_time_cross_80.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030_time_cross_80.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6951447 371.3   13077640 698.5 12131749 648.0
Vcells 16598641 126.7   31331508 239.1 26042924 198.7


In [23]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030_time_cross_80.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030_time_cross_80.Rds", sep = ""))

In [24]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

 [1] 10 32 35 36 45 53 62 68 73 83 91
[1] "----------"
 [1]   8  22  41  44  54  55  92  95  97 104
[1] "----------"
 [1]   2   4   9  24  48  61  67  70  89  99 102
[1] "----------"
 [1]  12  27  42  43  52  72  87  93 103 106
[1] "----------"
 [1]   7  16  17  37  50  51  64  66  74  88  90 105
[1] "----------"
 [1]  14  28  33  40  46  56  59  65  71 107
[1] "----------"
 [1]   5  19  20  25  34  39  60  79  81  86 100
[1] "----------"
 [1]   6  18  21  29  47  63  76  77  96  98 108
[1] "----------"
 [1]   3  11  23  30  31  38  58  78  80  82  94 101
[1] "----------"
 [1]  1 13 15 26 49 57 69 75 84 85
[1] "----------"


In [ ]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [ ]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  1 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  1 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  1 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.755431287728589 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  -0.0302479704498654 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.0798857148812623 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-3247837 : Last ARI =  1 : La

[1] "Last RXN_ID =  R-MMU-450580 : Last ARI =  0.923277651477711 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6791222 : Last ARI =  0.923277651477711 : Last ECOUNT =  61 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  1 : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8936481 : Last ARI =  1 : Last ECOUNT =  77 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9677051 : Last ARI =  1 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-5336403 : Last ARI =  1 : Last ECOUNT =  72 

[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  1 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-174224 : Last ARI =  1 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-3788708 : Last ARI =  1 : Last ECOUNT =  23 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-927830 : Last ARI =  0.886610984811044 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-448955 : Last ARI =  0.848814749780509 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89 % remaining..."
[1] "Last RXN_ID =  R-MMU-200652 : Last ARI =  0.0235106818846941 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  88.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9624798 : Last ARI =  0.0612284780

[1] "Last RXN_ID =  R-MMU-9013004 : Last ARI =  1 : Last ECOUNT =  16 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.1 % remaining..."
[1] "Last RXN_ID =  R-MMU-9818576 : Last ARI =  -0.0350644809592308 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84 % remaining..."
[1] "Last RXN_ID =  R-MMU-2029273 : Last ARI =  0.923277651477711 : Last ECOUNT =  71 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-8850356 : Last ARI =  0.812390908720907 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1363303 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-188386 : Last ARI =  0.889759645937545 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  83.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-390593 : Last 

[1] "Last RXN_ID =  R-MMU-1449633 : Last ARI =  1 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-211910 : Last ARI =  1 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928646 : Last ARI =  0.926909492273731 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-8848484 : Last ARI =  0.923034825870647 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9022185 : Last ARI =  0.818485651214128 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2032396 : Last ARI =  0.606155677535412 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  78.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-182990 : Last A

...

[1] "Last RXN_ID =  R-MMU-114683 : Last ARI =  0.625608601583693 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-75090 : Last ARI =  0.267679089295762 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-77342 : Last ARI =  0.164477278338685 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-449937 : Last ARI =  0.0472575140077618 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9718020 : Last ARI =  0.225449506472336 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6806966 : Last ARI =  0.45470118188722 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  22.3 % remaining..."
[1] "Last RXN_ID

[1] "Last RXN_ID =  R-NUL-2128982 : Last ARI =  0.0547357117920577 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9757010 : Last ARI =  0.959701492537313 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9653592 : Last ARI =  0.00237114573782546 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9668464 : Last ARI =  -0.0642188696101031 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2454118 : Last ARI =  0.337260102483774 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-159425 : Last ARI =  0.923034825870647 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  17.3 % remaining..."
[1] "L

[1] "Last RXN_ID =  R-MMU-9012036 : Last ARI =  -0.0459706541494071 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-975633 : Last ARI =  0.679440535090016 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9027532 : Last ARI =  0.769590128826068 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-917888 : Last ARI =  0.487408205141951 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6800315 : Last ARI =  0.615883991099154 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-200424 : Last ARI =  -0.00737708303618333 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  12.2 % remaining..."
[1] "Las

[1] "Last RXN_ID =  R-MMU-6809624 : Last ARI =  0.0619949958985371 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-192312 : Last ARI =  0.926183486997147 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8879117 : Last ARI =  0.386136424257645 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-139909 : Last ARI =  0.0634896884157098 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-8851979 : Last ARI =  0.283662844164071 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-9704248 : Last ARI =  0.596988490676323 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  7.2 % remaining..."
[1] "Last RXN_ID

[1] "Last RXN_ID =  R-MMU-5661125 : Last ARI =  0.932963988919668 : Last ECOUNT =  14 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9695949 : Last ARI =  0.642300653594771 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1855172 : Last ARI =  0.390744334391625 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-6807055 : Last ARI =  0.896540147860065 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.3 % remaining..."
[1] "Last RXN_ID =  R-MMU-1462039 : Last ARI =  0.5279750872098 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.2 % remaining..."
[1] "Last RXN_ID =  R-MMU-3341294 : Last ARI =  0.261436703177861 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  2.1 % remaining..."
[1] "Last RXN_ID 

In [27]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030_time_cross_80.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030_time_cross_80.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030_time_cross_80.Rds", sep = ""))

In [28]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030_time_cross_80.Rds", sep = ""))

                           RXN_ID MISCLASS          ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0  1.000000000     14
R-MMU-170666.30.00   R-MMU-170671        0  1.000000000     14
R-MMU-170671.0.00    R-MMU-170674        0  1.000000000     14
R-MMU-170671.30.00   R-MMU-170686        0  1.000000000     14
R-MMU-170674.0.00    R-MMU-380073        0  1.000000000    182
R-MMU-170674.30.00   R-MMU-392187        0  1.000000000     22
R-MMU-170686.0.00    R-MMU-392195        0  1.000000000     22
R-MMU-170686.30.00   R-MMU-392202        0  1.000000000     22
R-MMU-380073.0.00    R-MMU-749454        0  1.000000000    178
R-MMU-380073.30.00   R-MMU-749456        0  1.000000000    178
R-MMU-392187.0.00    R-MMU-751001        0  1.000000000     21
R-MMU-392187.30.00  R-MMU-8964229        0  1.000000000      5
R-MMU-392195.0.00   R-MMU-8964242        0  1.000000000      5
R-MMU-392195.30.00  R-MMU-8964329        0  1.000000000      5
R-MMU-392202.0.00   R-MMU-8982012        0  0.959701493

R-MMU-205112.30.00  R-MMU-5654397        0  0.959701493     16
R-MMU-205117.0.00   R-MMU-5654399        0  0.959701493     16
R-MMU-205117.30.00  R-MMU-5654402        0  0.959701493     20
R-MMU-209532.0.00   R-MMU-5654404        0  0.959701493     16
R-MMU-209532.30.00  R-MMU-5654406        0  0.959701493     18
R-MMU-209566.0.00   R-MMU-5654407        0  0.959701493     16
R-MMU-209566.30.00  R-MMU-5654408        0  0.959701493     12
R-MMU-9012761.0.00  R-MMU-5654409        0  0.959701493     12
R-MMU-9012761.30.00 R-MMU-5654413        0  1.000000000     17
R-MMU-3247837.0.00  R-MMU-5654416        0  0.959701493     15
R-MMU-3247837.30.00 R-MMU-5654418        0  1.000000000     14
R-MMU-3247840.0.00  R-MMU-5654422        0  1.000000000     14
R-MMU-3247840.30.00 R-MMU-5654423        0  1.000000000     16
R-MMU-3247843.0.00  R-MMU-5654425        0  1.000000000     14
R-MMU-3247843.30.00 R-MMU-5654426        0  1.000000000     18
R-MMU-3247844.0.00  R-MMU-5654428        0  1.000000000

R-MMU-5654397.30.00 R-MMU-9753126        0  0.708479018      5
R-MMU-5654399.0.00   R-MMU-514604        0  0.449485790      2
R-MMU-5654399.30.00  R-MMU-975389        0  0.721913282      4
R-MMU-5654402.0.00  R-MMU-1169394        0  0.959701493      5
R-MMU-5654402.30.00 R-MMU-1169398        0  0.835752865      5
R-MMU-5654404.0.00  R-MMU-1169402        0  0.895893759      6
R-MMU-5654404.30.00 R-MMU-1169403        0  0.626917672      4
R-MMU-5654406.0.00  R-MMU-1169405        0  0.885884980      5
R-MMU-5654406.30.00 R-MMU-1169406        0  1.000000000     12
R-MMU-5654407.0.00  R-MMU-1678842        0  1.000000000     13
R-MMU-5654407.30.00 R-MMU-1678843        0  0.847402985      5
R-MMU-5654408.0.00  R-MMU-5653754        0  1.000000000     21
R-MMU-5654408.30.00 R-MMU-5653756        0  1.000000000     19
R-MMU-5654409.0.00  R-MMU-9833973        0  0.768864124      5
R-MMU-5654409.30.00  R-MMU-426240        0  1.000000000     10
R-MMU-5654413.0.00   R-MMU-199050        0  0.959701493

R-MMU-5654664.30.00  R-MMU-389158        0  1.000000000      9
R-MMU-5654667.0.00   R-MMU-389348        0  0.608810938      8
R-MMU-5654667.30.00  R-MMU-389350        0  0.709092979      8
R-MMU-5654669.0.00   R-MMU-389352        0  0.680810938      7
R-MMU-5654669.30.00  R-MMU-389354        0  0.680810938      7
R-MMU-5654672.0.00   R-MMU-389381        0  0.680928585      6
R-MMU-5654672.30.00  R-MMU-389758        0  0.929411765     22
R-MMU-5654673.0.00   R-MMU-389759        0  0.963333333      7
R-MMU-5654673.30.00  R-MMU-389762        0  0.775401800      5
R-MMU-5654677.0.00   R-MMU-434836        0  0.704884246      3
R-MMU-5654677.30.00  R-MMU-452100        0  0.616878456      6
R-MMU-5654679.0.00  R-MMU-8852200        0  0.879104478     17
R-MMU-5654679.30.00 R-MMU-8855381        0  0.889113257     18
R-MMU-5654684.0.00  R-MMU-8857925        0  1.000000000     15
R-MMU-5654684.30.00 R-MMU-9706312        0  0.776370631      2
R-MMU-5654690.0.00  R-MMU-8849348        0  0.851277651

R-MMU-9753126.30.00  R-MMU-450580        0  0.923277651     12
R-MMU-514604.0.00     R-MMU-72103        0  0.959701493     43
R-MMU-514604.30.00  R-MMU-9770119        0  1.000000000     69
R-MMU-975389.0.00   R-MMU-9770141        0  1.000000000    123
R-MMU-975389.30.00  R-MMU-9770142        0  1.000000000    132
R-MMU-1169394.0.00  R-MMU-9770145        0  1.000000000    145
R-MMU-1169394.30.00 R-MMU-9770236        0  1.000000000    119
R-MMU-1169398.0.00  R-MMU-6798748        0  1.000000000    106
R-MMU-1169398.30.00 R-MMU-6800434        0  1.000000000    134
R-MMU-1169402.0.00  R-MMU-6791218        0  0.886610985     68
R-MMU-1169402.30.00 R-MMU-6791222        0  0.923277651     61
R-MMU-1169403.0.00  R-MMU-1605595        0  1.000000000      3
R-MMU-1169403.30.00 R-MMU-1605624        0  0.926183487      6
R-MMU-1169405.0.00  R-MMU-1605717        0  0.697342269      1
R-MMU-1169405.30.00 R-MMU-6798751        0  1.000000000    105
R-MMU-1169406.0.00  R-MMU-9840795        0  0.813884980

...

R-MMU-4551446.0.00   R-MMU-427570        0  0.708424563      2
R-MMU-4551446.30.00 R-MMU-2161526        0 -0.018800087      1
R-MMU-6807585.0.00  R-MMU-9659680        0 -0.018800087      1
R-MMU-6807585.30.00 R-MMU-9759454        0 -0.018800087      1
R-MMU-390912.0.00   R-MMU-9759461        0 -0.018800087      1
R-MMU-390912.30.00   R-MMU-372819        0  0.456253029      1
R-MMU-9692539.0.00  R-MMU-5606210        0  0.929815328      3
R-MMU-9692539.30.00 R-MMU-5606256        0  0.929815328      3
R-MMU-202709.0.00   R-MMU-5629230        0  0.929815328      4
R-MMU-202709.30.00  R-MMU-8942575        0  0.364139022      2
R-MMU-1254248.0.00  R-MMU-8953339        0  0.213882405      1
R-MMU-1254248.30.00 R-MMU-5696839        0 -0.021676633      1
R-MMU-1254251.0.00  R-MMU-6801456        0 -0.021676633      1
R-MMU-1254251.30.00 R-MMU-8944219        0  0.611894875      1
R-MMU-2032770.0.00   R-MMU-879594        0  0.594289400      1
R-MMU-2032770.30.00  R-MMU-211874        0  0.307073645

R-MMU-9603982.0.00   R-MMU-391942        0  0.435845863      1
R-MMU-9603982.30.00 R-MMU-9726949        0  0.435845863      1
R-MMU-9603983.0.00  R-MMU-1564169        0  0.437354054      3
R-MMU-9603983.30.00 R-MMU-1602473        0  0.583752114      1
R-MMU-168412.0.00    R-MMU-389424        0  0.065853868      1
R-MMU-168412.30.00  R-MMU-9014672        0  0.025076365      2
R-MMU-708349.0.00   R-MMU-9014678        0  0.025076365      2
R-MMU-708349.30.00   R-MMU-162873        0  0.382948124      1
R-MMU-6798474.0.00  R-MMU-8878914        0  0.244854771      1
R-MMU-6798474.30.00 R-MMU-8858270        0  0.860183487      1
R-MMU-6798528.0.00  R-MMU-5605262        0  0.073631911      1
R-MMU-6798528.30.00  R-MMU-444620        0  0.535953665      2
R-MMU-9844860.0.00  R-MMU-9009950        0  0.034494221      1
R-MMU-9844860.30.00 R-MMU-8937744        0  0.466984697      1
R-MMU-446185.0.00   R-MMU-8937767        0  0.466984697      1
R-MMU-446185.30.00   R-MMU-914067        0  0.587778844

R-MMU-1461971.0.00  R-MMU-5336430        0  0.529518463      2
R-MMU-1461971.30.00 R-MMU-5687126        0  0.041831406      1
R-MMU-1461982.0.00  R-MMU-9624527        0  0.041831406      1
R-MMU-1461982.30.00 R-MMU-9699581        0  0.041831406      1
R-MMU-1461995.0.00  R-MMU-5619439        0  0.272501416      1
R-MMU-1461995.30.00  R-MMU-417896        0  0.735425735      1
R-MMU-1462003.0.00  R-MMU-9634937        0  0.001313854      1
R-MMU-1462003.30.00  R-MMU-381608        0  0.071138521      1
R-MMU-1462005.0.00  R-MMU-8959510        0  0.879104478      1
R-MMU-1462005.30.00 R-MMU-9034714        0  0.510573645      2
R-MMU-1462014.0.00  R-MMU-9615249        0  0.419194907      1
R-MMU-1462014.30.00 R-MMU-3296326        0  0.335586091      1
R-MMU-1462039.0.00  R-MMU-5205798        0  0.966481994     15
R-MMU-1462039.30.00 R-MMU-5205820        0  0.882979144     20
R-MMU-1462041.0.00  R-MMU-5229203        0  0.882979144     20
R-MMU-1462041.30.00 R-MMU-5661126        0  0.966481994

R-MMU-77338.0.00    R-MMU-9663768        0  0.815779924      1
R-MMU-77338.30.00   R-MMU-9749977        0  0.923034826      2
R-MMU-77345.0.00     R-MMU-266204        0  1.000000000      2
R-MMU-77345.30.00    R-MMU-419334        0 -0.057621723      1
R-MMU-1237160.0.00  R-MMU-5638333        0  0.785653405     14
R-MMU-1237160.30.00 R-MMU-5649800        0  0.785653405     14
R-MMU-6810376.0.00  R-MMU-5649802        0  0.785653405     14
R-MMU-6810376.30.00 R-MMU-9696274        0  0.286844655      1
R-MMU-5674385.0.00   R-MMU-109380        0  0.578107202      1
R-MMU-5674385.30.00  R-MMU-109387        0  0.578107202      1
R-MMU-9610166.0.00  R-NUL-2974720        0  0.659998334      1
R-MMU-9610166.30.00 R-MMU-6811479        0  0.737888636      3
R-MMU-2408505.0.00  R-MMU-6811508        0  0.737888636      3
R-MMU-2408505.30.00 R-MMU-9689680        0  0.603661507      1
R-MMU-3446785.0.00  R-NUL-9025041        0  0.047272478      1
R-MMU-3446785.30.00 R-NUL-9025059        0  0.047272478

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030_time_cross_80.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030_time_cross_80.txt",sep=""), header=FALSE)
print(sid)

            V1
1   SRR7817623
2   SRR7817626
3   SRR7817614
4   SRR7817675
5   SRR1636593
6   SRR7817627
7   SRR3593526
8   SRR7817639
9   SRR7817615
10  SRR7817684
11  SRR4317608
12  SRR4317607
13  SRR4317612
14  SRR7817648
15  SRR7817674
16  SRR7817647
17  SRR7817636
18  SRR7817659
19  SRR7817671
20  SRR4317610
21  SRR1636591
22  SRR7817685
23  SRR7817664
24  SRR7817611
25  SRR3593525
26  SRR4317609
27  SRR7817635
28  SRR3593527
29  SRR7817661
30  SRR7817676
31  SRR7817673
32  SRR7817698
33  SRR7817683
34  SRR7817612
35  SRR7817613
36  SRR7817686
37  SRR7817662
38  SRR7817640
39  SRR7817616
40  SRR1636588
41  SRR7817624
42  SRR7817637
43  SRR1636586
44  SRR7817660
45  SRR1636589
46  SRR7817663
47  SRR7817695
48  SRR7817628
49  SRR7817651
50  SRR7817649
51  SRR7817625
52  SRR7817697
53  SRR4317611
54  SRR3593523
55  SRR3593578
56  SRR1636674
57  SRR7817667
58  SRR7817618
59  SRR7817617
60  SRR3593575
61  SRR7817658
62  SRR7817679
63  SRR7817642
64  SRR7817644
65  SRR3593576
66  SRR781

In [ ]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  
     colors = c("red", "green"),  # 定义颜
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  
)
saveWidget(p, "plotly_chart030_time_cross_80.html", selfcontained = TRUE)